# Explore GRS images

## Load python libraries

In [ ]:

import glob
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as xrio # activate the rio accessor

#import cartopy
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import locale
#locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('TkAgg')
import hvplot.xarray

import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews import opts
hv.extension('bokeh')
import datashader as ds
from datashader import transfer_functions as tf 
import colorcet as cc
import panel as pn
import panel.widgets as pnw
#import ipywidgets as widgets

import pyproj as ppj
import rasterio
from affine import Affine
from shapely.geometry import box,Point, mapping
from shapely.ops import transform


import grstbx
from grstbx import visual

u = grstbx.utils
opj = os.path.join

grstbx.__version__

In [ ]:
pn.__version__

## Set Dask local cluster

In [ ]:
#import dask.distributed
#cluster = dask.distributed.LocalCluster()
#client = dask.distributed.Client(cluster)
#cluster.dashboard_link

## Set PROJ path if necessary

In [ ]:
ppj.datadir.get_data_dir()
#ppj.datadir.set_data_dir('/work/scratch/harmelt/envs/grstbx/share/proj')

## Set the images you want to play with

In [ ]:
file = '/sat_data/satellite/sentinel2/L3/test_abhe_2022_nov-dec.nc'#]
file = '/home/harmel/Dropbox/Dropbox/satellite/S2/datacube/31TGM_2023-06_leman.nc'
raster = xr.open_dataset(file, decode_coords='all').set_coords('spatial_ref')
dc =grstbx.l2grs(file)
dc.raster=raster
dc.raster 

In [ ]:
dc.raster.rio.crs #.rio.crs#.rio.write_crs(3857)


## **Fast checking of the RGB images**

In [ ]:
bands=[4,2,1]
#bands=[3,2,1]
coarsening = 1

gamma=1
fig = (dc.raster.Rrs.isel(wl=bands)[:,:,::coarsening, ::coarsening]**(1/gamma)).plot.imshow(col='time', col_wrap=4,robust=True)
for ax in fig.axes.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


## **Check spectral datacube** (i.e., Remote Sensing Reflectance, R<sub>rs</sub>, sr<sup>-1</sup>)

To quickly check your data visually, you can use the *visual* module of *grstbx*. Please use the polygon tool to select and extract you region of interest for further analyses.

In [ ]:

v=visual.view_spectral(dc.raster.Rrs)
v.visu()

In [ ]:
geom_ = v.get_geom(v.aoi_stream,crs=dc.raster.rio.crs)
Rrs_clipped = dc.raster.Rrs.rio.clip(geom_.geometry.values)

In [ ]:
bands=[4,2,1]
bands=[3,2,1]

fig = Rrs_clipped.isel(wl=bands).plot.imshow(col='time', col_wrap=4,robust=True)
for ax in fig.axes.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


## **Check water quality parameters** (e.g., Chl-a concentration from diverse "algorithms")

## First construct datacube of parameters
Choose between full image or clipped one:

In [ ]:
# if full image:
Rrs = dc.raster.Rrs
# or extracted region:
#Rrs = Rrs_clipped

### Check blue over green ratio for Chl retrieval with OC2 from NASA
$log_{10}(chlor\_a) = a_0 + \sum\limits_{i=1}^4 a_i \left(log_{10}\left(\frac{R_{rs}(\lambda_{blue})}{R_{rs}(\lambda_{green})}\right)\right)^i$

In [ ]:

# NASA OC2 fro MODIS; bands 488, 547 nm
a = [0.2500,-2.4752,1.4061,-2.8233,0.5405]
# NASA OC2 for OCTS; bands 490, 565 nm
a = [0.2236,-1.8296,1.9094,-2.9481,-0.1718]

ratio = np.log10(Rrs.isel(wl=1)/Rrs.isel(wl=2))
logchl=0
for i in range(len(a)):
    logchl+=a[i]*ratio**i
chl = 10**(logchl)
chl.name='Chl-a_OC2'
chl.attrs['units']='mg.m-3'
chl.attrs['description']= 'Chl-a concentration from NASA OC2 with OCTS parameterization, bands 490, 565 nm',
chl = chl.where((chl >= 0) & (chl <= 60))


### CDOM retrieval based on Brezonik et al, 2015


In [ ]:
a = [1.872,-0.83]
acdom = np.exp(a[0] + a[1] * np.log(Rrs.isel(wl=1)/Rrs.isel(wl=5)))
acdom.name='a_cdom_b2015'
acdom.attrs['units']='m-1'
acdom.attrs['description']='CDOM absorption at 440 nm-1'
acdom= acdom.where((acdom >= 0) & (acdom <= 10))


### Total suspended particulate matter (SPM) from Nechad et al., 2010, 2016 formulation
spm in mg/L

In [ ]:
a = [610.94*np.pi, 0.2324/np.pi]
Rrs_ = Rrs.isel(wl=3)
spm = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm.name='SPM_N2016'
spm.attrs['units']='mg/L'
spm.attrs['description']='Concentration of suspended particulate matter from band 665 nm'
spm= spm.where((spm >= 0) & (spm <= 150))


In [ ]:
watqual = xr.merge([chl,acdom,spm])

In [ ]:
v2b=visual.view_param(watqual)
v2b.visu()




# Further analyses and time series

In [ ]:
stacked = Rrs_clipped.dropna('time',thresh=0).stack(gridcell=["y", "x"]).dropna('gridcell',thresh=0)


In [ ]:
group_coord ='wl'
stat_coord='gridcell'
stats = xr.Dataset({'median':stacked.groupby(group_coord).median(stat_coord)})
stats['q25'] = stacked.groupby(group_coord).quantile(0.25,dim=stat_coord)
stats['q75'] = stacked.groupby(group_coord).quantile(0.75,dim=stat_coord)
stats['min'] = stacked.groupby(group_coord).min(stat_coord)
stats['max'] = stacked.groupby(group_coord).max(stat_coord)
stats['mean'] = stacked.groupby(group_coord).mean(stat_coord)
stats['std'] = stacked.groupby(group_coord).std(stat_coord)
stats['pix_num'] = stacked.count(stat_coord)

In [ ]:
%matplotlib inline
num_items = len(stats.time)
col_wrap=4
rows=int(np.ceil(num_items/col_wrap))

fig, axs = plt.subplots(nrows=rows,ncols=col_wrap, sharex=True,sharey=True, figsize=(20, rows*3.5))#,sharey=True
fig.subplots_adjust(hspace=0.1,wspace=0.1)
axs_ = axs.ravel()
[axi.set_axis_off() for axi in axs_]
for iax,(_,group) in enumerate(stats.groupby('time')): 
    date = group.time.dt.date.values
    
    axs_[iax].set_axis_on()
    axs_[iax].axhline(y=0,color='k',lw=1)
    axs_[iax].plot(group.wl,group['median'],c='k')
    axs_[iax].plot(group.wl,group['mean'],c='red',ls='--')
    axs_[iax].fill_between(group.wl, group['q25'], group['q75'],alpha=0.3,color='grey')
    axs_[iax].set_title(date)
plt.show()

In [ ]:
import geopandas as gpd
def get_buffer_box(p_lat, p_long, distance_m):
    # distance is d/2 of the square buffer around the point,
    # from center to corner;
    # find buffer width in meters
    buffer_width_m = distance_m  / np.sqrt(2)

    # EPSG:4326 sets Coordinate Reference System to WGS84 to match input
    wgs84_pt_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p_long], [p_lat], crs='4326'))

    # find suitable projected coordinate system for distance
    utm_crs = wgs84_pt_gdf.estimate_utm_crs()
    # reproject to UTM -> create square buffer (cap_style = 3) around point -> reproject back to WGS84
    wgs84_buffer = wgs84_pt_gdf.to_crs(utm_crs).buffer(buffer_width_m, cap_style=3).to_crs('EPSG:4326')
    # wgs84_buffer.bounds returns bounding box as pandas dataframe,
    # .values[0] will extract first row as an array
    return wgs84_buffer#.bounds

In [ ]:
#set location, geom coordinate system and extent
lat,lon= 46.5002275, 6.6608353
extent=100
# set subset (Area of interest)
buffer = get_buffer_box(lat, lon, extent)

# wkt = misc.wktbox(lon, lat,width=width,height=height)
lon_min, lat_min, lon_max, lat_max = buffer.bounds.values[0]

print(buffer.bounds)

In [ ]:
minx, miny, maxx, maxy = buffer.to_crs(3857).bounds.values[0]
print(minx, miny, maxx, maxy)

In [ ]:
clipped = dc.raster.Rrs.rio.clip_box(minx, miny, maxx, maxy)

In [ ]:
clipped

# Check surface rugosity via sunglint BRDF

In [ ]:

raster = dc.raster.BRDFg#.isel(time=-1,drop=True)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'gray',colorbar=True)#.hist(bin_range=(0,0.02) ) 
widget = pn.widgets.RangeSlider(start=0, end=0.1,step=0.0005)

jscode = """
    color_mapper.low = cb_obj.value[0];
    color_mapper.high = cb_obj.value[1];
"""
link = widget.jslink(im, code={'value': jscode})
pn.Column(widget, im)

In [ ]:

shaded = []
for name, raster in dc.raster.BRDFg.groupby('time'):
    img = tf.shade(raster.squeeze()[::-1],cmap=cc.gray)
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 3
imgs

In [ ]:
visual.image_viewer().param_date(spm,cmap='bgyw')

In [ ]:

shaded = []
for name, raster in dc.Rrs.Rrs.isel(time=-1).groupby('wl'):
    img = tf.shade(raster,cmap=cc.kbc)
    img.name = '{:.2f}'.format(name)+' nm'
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 5
imgs

In [ ]:
shaded = []
for name, raster in BRDFg.groupby('time'):
    img = tf.shade(raster,cmap=cc.gray, span=(0,0.025),how='log')
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 4
imgs

# Play with time series

In [ ]:
raster = chl

param = raster.name
third_dim = 'time'
time= raster.time.data
Ntime = len(time)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'RdBu_r',colorbar=True,clim=(0,100))#.hist(bin_range=(0,0.02) ) 

polys = hv.Polygons([])
box_stream = hv.streams.BoxEdit(source=polys)
dmap, dmap_std=[],[]

def roi_curves(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'time', param)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.nanmean).data
        print(mean)
        #std = selection.aggregate(third_dim, np.std).data
        time = mean[third_dim]

        curves[i]= hv.Curve((time,mean[param]),'time', param) 

    return hv.NdOverlay(curves)


# a bit dirty to have two similar function, but holoviews does not like mixing Curve and Spread for the same stream
def roi_spreads(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'time', param)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.nanmean).data
        std = selection.aggregate(third_dim, np.nanstd).data
        time = mean[third_dim]

        curves[i]=  hv.Spread((time,mean[param],std[param]),fill_alpha=0.3)

    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    


In [ ]:
# visualize and play
graphs = ((mean*std ).relabel(param))
layout = (im * polys +graphs    ).opts(
    opts.Curve(width=1200,height=600, framewise=True), 
    opts.Polygons(fill_alpha=0.2, color='green',line_color='black'), 
    ).cols(1)
layout 